# Think Bayes

Second Edition

Copyright 2020 Allen B. Downey

License: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [4]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from empiricaldist import Pmf
from utils import decorate, savefig

## The train problem

The train problem more hypothese than the dice problem.

In [6]:
hypos = np.arange(1, 1001)
prior = Pmf(1, hypos)

But the update function is the same.

In [12]:
def update_train(pmf, data):
    """Update a pmf based on new data.
    
    pmf: Pmf of possible dice and their probabilities
    data: integer train #
    """
    hypos = pmf.qs
    likelihood = 1 / hypos
    impossible = (data > hypos)
    likelihood[impossible] = 0
    pmf *= likelihood
    pmf.normalize()

But there are many more hypotheses

In [8]:
data = 60
posterior = prior.copy()
update_train(posterior, data)

Here's what the posterior looks like

In [13]:
posterior.plot(label='Posterior after train 60')
decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Train Problem')
savefig('fig04-01')

Here's how we can compute the posterior mean

$ \mathrm{mean} = \sum_i ~ p_i q_i $

In [10]:
np.sum(posterior.ps * posterior.qs)

Or we can use the method provided by `Pmf`:

In [11]:
posterior.mean()

## Sensitivity to the prior

Here's a function that solves the train problem for different priors and data

In [17]:
def update_train(pmf, dataset):
    """Solves the train problem.
    
    pmf: 
    dataset: sequence of observed train numbers
    
    returns: Pmf representing the posterior distribution
    """
    for data in dataset:
        pmf *= likelihood_train(data, hypos)

    pmf.normalize()

Let's run it with the same dataset and several uniform priors

In [18]:
dataset = [30, 60, 90]

for high in [500, 1000, 2000]:
    hypos = np.arange(1, high+1)
    pmf = Pmf(1, hypos)
    for data in dataset:
        update_train(pmf, data)
    print(high, pmf.mean())

The results are quite sensitive to the prior, even with several observations.

## Power law prior

Now let's try it with a power law prior.

In [35]:
hypos = np.arange(1, 1001)
uniform = Pmf(1, hypos, name='uniform')
uniform.normalize()

In [36]:
alpha = 1.0
ps = hypos**(-alpha)
power = Pmf(ps, hypos, name='power law')
power.normalize()

Here's what a power law prior looks like, compared to a uniform prior

In [37]:
uniform.plot()
power.plot()

decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Train Problem')

Now let's see what the posteriors look like after observing one train.

In [38]:
dataset = [60]

update_train(uniform, dataset)
update_train(power, dataset)

In [39]:
uniform.plot()
power.plot()

decorate(xlabel='Number of trains',
         ylabel='PMF',
         title='Train Problem')
savefig('fig04-02')

The power law gives less prior probability to high values, which yields lower posterior means, and less sensitivity to the upper bound.

In [40]:
alpha = 1.0
dataset = [30, 60, 90]

for high in [500, 1000, 2000]:
    hypos = np.arange(1, high+1)
    ps = hypos**(-alpha)
    power = Pmf(ps, hypos)
    for data in dataset:
        update_train(power, data)
    print(high, power.mean())

## Credible intervals

To compute credible intervals, we can use the `credible_interval` method on the posterior.

In [41]:
alpha = 1.0
dataset = [30, 60, 90]

hypos = np.arange(1, 1001)
ps = hypos**(-alpha)
power = Pmf(ps, hypos)
for data in dataset:
    update_train(power, data)

power.credible_interval(0.9)

In [125]:
power.lt_dist(100)

In [126]:
def quantile(pmf, prob):
    """Compute a quantile.
    
    pmf: Pmf representing a normalized distribution
    prob: float probability
    
    returns: quantity from the distribution
    """
    total = 0
    for q, p in pmf.items():
        total += p
        if total >= prob:
            return q
    return np.nan

For example, the median is 113.

In [129]:
quantile(power, 0.5)

We can compute the 5th and 95th percentiles, which make up a 90% confidence interval.

In [130]:
quantile(power, 0.05)

In [131]:
quantile(power, 0.95)

`Pmf` provides `quantile` which does the same thing.

In [42]:
power.quantile([0.05, 0.95])

It also provides `credible_interval` which computes an interval that contains the given probability.

In [132]:
power.credible_interval(0.9)

## Exercises

**Exercise:** Suppose you are giving a talk in a large lecture hall and you want to estimate the number of people in the audience.  There are too many to count, so you ask how many people were born on May 11 and two people raise their hands.  You ask how many were born on May 23 and 1 person raises their hand.  Finally, you ask how many were born on August 1, and no one raises their hand.

How many people are in the audience?  What is the 90% credible interval for your estimate?

Hint: Remember the binomial distribution.

In [117]:
# Solution goes here

In [118]:
# Solution goes here

In [119]:
# Solution goes here

In [120]:
# Solution goes here

In [121]:
# Solution goes here

In [123]:
# Solution goes here

**Exercise:** I often see [rabbits](https://en.wikipedia.org/wiki/Eastern_cottontail) in the garden behind my house, but it's not easy to tell them apart, so I don't really know how many there are.

Suppose I deploy a motion-sensing [camera trap](https://en.wikipedia.org/wiki/Camera_trap) that takes a picture of the first rabbit it sees each day.  After three days, I compare the pictures and conclude that two of them are the same rabbit and the other is different.

How many rabbits visit my garden?

To answer this question, we have to think about the prior distribution and the likelihood of the data:

* I have sometimes seen four rabbits at the same time, so I know there are at least that many.  I would be surprised if there were more than 10.  So, at least as a starting place, I think a uniform prior from 4 to 10 is reasonable.

* To keep things simple, let's assume that all rabbits who visit my garden are equally likely to be caught by the camera trap in a given day.  Let's also assume it is guaranteed that the camera trap gets a picture every day.

In [47]:
# Solution goes here

In [50]:
# Solution goes here

In [54]:
# Solution goes here

**Exercise:** Suppose that in the criminal justice system, all prison sentences are either 1, 2, or 3 years, with an equal number of each.  One day, you visit a prison and choose a prisoner at random.  What is the probability that they are serving a 3-year sentence?  What is the average remaining sentence of the prisoners you observe?

In [80]:
# Solution goes here

In [76]:
# Solution goes here

In [78]:
# Solution goes here

**Exercise:** If I chose a random adult in the U.S., what is the probability that they have a sibling?  To be precise, what is the probability that their mother has had at least one other child.

[This article from the Pew Research Center](https://www.pewsocialtrends.org/2015/05/07/family-size-among-mothers/) provides some relevant data.  From it, I extracted the following distribution of family size for mothers in the U.S. who were 40-44 years old in 2014:

In [92]:
qs = [1, 2, 3, 4]
ps = [22, 41, 24, 14]
prior = Pmf(ps, qs)
prior.bar(alpha=0.7)

plt.xticks(qs, ['1 child', '2 children', '3 children', '4+ children'])
decorate(ylabel='PMF',
         title='Distribution of family size')

For simplicity, let's assume that all families in the 4+ category have exactly 4 children.

In [94]:
# Solution goes here

In [95]:
# Solution goes here

**Exercise:** The [Doomsday argument](https://en.wikipedia.org/wiki/Doomsday_argument) is "a probabilistic argument that claims to predict the number of future members of the human species given an estimate of the total number of humans born so far."

Suppose there are only two kinds of intelligent civilizations that can happen in the universe.  The "short-lived" kind go exinct after only 200 billion individuals are born.  The "long-lived" kind survive until 2,000 billion individuals are born.
And suppose that the two kinds of civilization are equally likely.
Which kind of civilization do you think we live in?  

The Doomsday argument says we can use the total number of humans born so far as data.
According to the [Population Reference Bureau](https://www.prb.org/howmanypeoplehaveeverlivedonearth/), the total number of people who have ever lived is about 108 billion.

Since you were born quite recently, let's assume that you are, in fact, human being number 108 billion.
If $N$ is the total number who will ever live and we consider you to be a randomly-chosen person, it is equally likely that you could have been person 1, or $N$, or any number in between.
So what is the probability that you would be number 108 billion?

Given this data and dubious prior, what is the probability that our civilization will be short-lived?

In [96]:
# Solution goes here

In [97]:
# Solution goes here

In [98]:
# According to this analysis, the probability is about 91% that our civilization will be short-lived.  
# But this conclusion is based on a dubious prior.

# And with so little data, the posterior depends strongly on the prior.  
# To see that, run this analysis again with a different prior, and see what the results look like.

# What do you think of the Doomsday argument?